In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('Data_for_UCI_named.csv')
df.head(8)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable
5,6.999209,9.109247,3.784066,4.267788,4.429669,-1.857139,-0.670397,-1.902133,0.261793,0.077930,0.542884,0.469931,-0.017385,stable
6,6.710166,3.765204,6.929314,8.818562,2.397419,-0.614590,-1.208826,-0.574004,0.177890,0.397977,0.402046,0.376630,0.005954,unstable
7,6.953512,1.379125,5.719400,7.870307,3.224495,-0.748998,-1.186517,-1.288980,0.371385,0.633204,0.732741,0.380544,0.016634,unstable


In [3]:
df = df.drop(columns='stab')
df.head(5)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [4]:
df.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [5]:
dummy = pd.get_dummies(df['stabf'])
df = pd.concat((df, dummy), axis=1)


marking stable= 1 and unstable = 0 

In [6]:
df = df.drop(columns=['stabf', 'unstable'])

In [7]:
df = df.rename(columns={'stable':'stabf'})

In [8]:
x = df.drop(columns='stabf')
y = df['stabf']

In [9]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2 , random_state= 1 ) 

In [10]:
x_train.shape
x_test.shape
y_train.shape
y_test.shape

(2000,)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_normalised = scaler.fit_transform(x_train)
x_test_normalised = scaler.fit_transform(x_test)
y_train.dtype

dtype('uint8')

In [29]:
import xgboost as xgb
import lightgbm as lgb

In [30]:
xgb_model = xgb(random_state= 1)
xgb_model.fit(x_train_normalised, y_train)
x_model = xgb_model.predict(x_test_normalised)
xg_score = xgb_model.score(x_test_normalised, y_test)
xg_score

TypeError: 'module' object is not callable

In [19]:
from sklearn.ensemble import RandomForestRegressor
forest_model = RandomForestRegressor(random_state= 1)
forest_model.fit(x_train_normalised, y_train)
forest_pred = forest_model.predict(x_test_normalised)
forestscore = forest_model.score(x_test_normalised, y_test)
forestscore


0.7201148021494871

In [21]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
accuracy = accuracy_score(y_true=y_test, y_pred= forest_pred)
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 4 )) 

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [17]:
from sklearn.ensemble import ExtraTreesClassifier
E_trees_model = ExtraTreesClassifier(random_state=1)


E_trees_model.fit(x_train_normalised, y_train)
E_trees_pred = E_trees_model.predict(x_test_normalised)
E_trees_pred


array([0, 0, 1, ..., 1, 0, 0], dtype=uint8)